In [2]:
# Nạp các gói thư viện NumPy và Pandas
import pandas as pd
import numpy as np
import regex
import string
import os
os.chdir(path)

# Nap du lieu media

In [3]:
title = pd.read_csv("data/NLP.csv")
title['date'] = pd.to_datetime(title['date'])
title

date  hour ma_ck                                              title  \
0   2023-02-24  17.5   ABC  ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán ...   
1   2023-02-24  17.2   ABC  ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng...   
2   2023-02-20  15.3   ABC  ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...   
3   2023-01-31  17.2   ABC             ABC: Báo cáo quản trị công ty năm 2022   
4   2023-01-30  15.5   ACM  ACM: Đính chính Báo cáo quản trị công ty năm 2022   
..         ...   ...   ...                                                ...   
816 2023-01-18   8.1   X77             X77: Báo cáo quản trị công ty năm 2022   
817 2023-02-09  13.2   XPH  XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...   
818 2023-02-09  13.2   XPH  XPH: Quyết định duy trì diện bị hạn chế giao d...   
819 2023-02-03   8.5   XPH  XPH: Giải trình liên quan đến Báo cáo tài chín...   
820 2023-02-02  13.4   XPH             XPH: Báo cáo quản trị công ty năm 2022   

       weekday                                          title_new  
0       Friday                    miễn_nhiệm dương kế_toán_trưởng  
1       Friday                       tham_dự đại_hội_đồng cổ_đông  
2       Monday  bổ_nhiệm hồ thị hà giữ chức người công_bố thôn...  
3      Tuesday                           báo_cáo quản_trị công_ty  
4       Monday                        đính_chính báo_cáo quản_trị  
..         ...                                                ...  
816  Wednesday                           báo_cáo quản_trị công_ty  
817   Thursday  bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ...  
818   Thursday  quyết_định duy_trì diện bị hạn_chế giao_dịch h...  
819     Friday     giải_trình liên_quan báo_cáo tài_chính niên độ  
820   Thursday                           báo_cáo quản_trị công_ty  

[821 rows x 6 columns]

# Nap du lieu gia chung khoan

In [4]:
price_root = pd.read_excel('data/price07_01_2023.xlsx', usecols = ['Date', 'maCK', 'Open', 'Close'])

In [5]:
price = price_root.copy()
price['Date'] = pd.DatetimeIndex(price['Date'], dayfirst=True)
price = price.rename(columns={'Date':'date',
                              'maCK':'ma_ck',
                              'Open':'open_price',
                              'Close':'close_price'})
price = price.sort_values(['ma_ck', 'date'], ascending=True)
price['open_price'] = price['open_price'].astype('str')
price['open_price'] = price['open_price'].str.replace('00.0','000')
price['open_price'] = price['open_price'].str.replace(',', '.')
# price = price[price['open_price'].isnull()==False]
price

<ipython-input-5-43a466ad553d>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  price['open_price'] = price['open_price'].str.replace('00.0','000')


date ma_ck open_price  close_price
30   2023-01-09   ABC        6.2          6.2
29   2023-01-10   ABC        6.2          6.2
28   2023-01-11   ABC        6.2          6.2
27   2023-01-12   ABC        6.2          6.2
26   2023-01-13   ABC        5.5          6.5
...         ...   ...        ...          ...
6791 2023-02-21   XPH        8.6          8.6
6790 2023-02-22   XPH        8.6          8.6
6789 2023-02-23   XPH        8.6          8.6
6788 2023-02-24   XPH        8.4          8.4
6787 2023-02-27   XPH        8.4          8.4

[6818 rows x 4 columns]

In [6]:
# Tam bo PVX va PXT
from tabulate import tabulate
price['open_price'] = price['open_price'].astype('float', errors='raise')
price = price.drop_duplicates(ignore_index=True)
print(tabulate(price.head(10), headers='keys'))

    date                 ma_ck      open_price    close_price
--  -------------------  -------  ------------  -------------
 0  2023-01-09 00:00:00  ABC               6.2            6.2
 1  2023-01-10 00:00:00  ABC               6.2            6.2
 2  2023-01-11 00:00:00  ABC               6.2            6.2
 3  2023-01-12 00:00:00  ABC               6.2            6.2
 4  2023-01-13 00:00:00  ABC               5.5            6.5
 5  2023-01-16 00:00:00  ABC               6.3            6.3
 6  2023-01-17 00:00:00  ABC               6.3            6.3
 7  2023-01-18 00:00:00  ABC               6.3            6.3
 8  2023-01-19 00:00:00  ABC               6.3            6.3
 9  2023-01-27 00:00:00  ABC               6.3            7.2


# Xu ly du lieu chung khoan

In [9]:
price['close_price_lag'] = price.groupby('ma_ck')['close_price'].shift()
price['open_price_next'] = price.groupby('ma_ck')['open_price'].shift(-1)
price = price.dropna()
price

date ma_ck  open_price  close_price  return_in_session  \
1    2023-01-10   ABC         6.2          6.2           0.000000   
2    2023-01-11   ABC         6.2          6.2           0.000000   
3    2023-01-12   ABC         6.2          6.2           0.000000   
4    2023-01-13   ABC         5.5          6.5           0.167054   
5    2023-01-16   ABC         6.3          6.3           0.000000   
...         ...   ...         ...          ...                ...   
6812 2023-02-20   XPH         8.6          8.6           0.000000   
6813 2023-02-21   XPH         8.6          8.6           0.000000   
6814 2023-02-22   XPH         8.6          8.6           0.000000   
6815 2023-02-23   XPH         8.6          8.6           0.000000   
6816 2023-02-24   XPH         8.4          8.4           0.000000   

      close_price_lag  open_price_next  
1                 6.2              6.2  
2                 6.2              6.2  
3                 6.2              5.5  
4                 6.2              6.3  
5                 6.5              6.3  
...               ...              ...  
6812              8.6              8.6  
6813              8.6              8.6  
6814              8.6              8.6  
6815              8.6              8.4  
6816              8.6              8.4  

[6378 rows x 7 columns]

## Tinh volability cua chung khoan

In [10]:
price['return_in_session'] = np.log(price['close_price']/price['open_price'])
price['return_before_session'] = np.log(price['open_price']/price['close_price_lag'])
price

<ipython-input-10-8a13868d1ab1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['return_in_session'] = np.log(price['close_price']/price['open_price'])
<ipython-input-10-8a13868d1ab1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['return_before_session'] = np.log(price['open_price']/price['close_price_lag'])


date ma_ck  open_price  close_price  return_in_session  \
1    2023-01-10   ABC         6.2          6.2           0.000000   
2    2023-01-11   ABC         6.2          6.2           0.000000   
3    2023-01-12   ABC         6.2          6.2           0.000000   
4    2023-01-13   ABC         5.5          6.5           0.167054   
5    2023-01-16   ABC         6.3          6.3           0.000000   
...         ...   ...         ...          ...                ...   
6812 2023-02-20   XPH         8.6          8.6           0.000000   
6813 2023-02-21   XPH         8.6          8.6           0.000000   
6814 2023-02-22   XPH         8.6          8.6           0.000000   
6815 2023-02-23   XPH         8.6          8.6           0.000000   
6816 2023-02-24   XPH         8.4          8.4           0.000000   

      close_price_lag  open_price_next  return_before_session  
1                 6.2              6.2               0.000000  
2                 6.2              6.2               0.000000  
3                 6.2              5.5               0.000000  
4                 6.2              6.3              -0.119801  
5                 6.5              6.3              -0.031253  
...               ...              ...                    ...  
6812              8.6              8.6               0.000000  
6813              8.6              8.6               0.000000  
6814              8.6              8.6               0.000000  
6815              8.6              8.4               0.000000  
6816              8.6              8.4              -0.023530  

[6378 rows x 8 columns]

In [11]:
price_stack = price[['date', 'ma_ck', 'return_in_session', 'return_before_session']].set_index(['date', 'ma_ck']).stack().reset_index()
price_stack.columns = ['date', 'ma_ck', 'session', 'return']
price_stack = price_stack.sort_values(['ma_ck', 'date', 'session'])
price_stack = price_stack[price_stack['return']!=np.inf]
price_stack = price_stack[price_stack['return']!=-np.inf]
print(tabulate(price_stack.head(10), headers='keys'))

    date                 ma_ck    session                    return
--  -------------------  -------  ---------------------  ----------
 1  2023-01-10 00:00:00  ABC      return_before_session   0
 0  2023-01-10 00:00:00  ABC      return_in_session       0
 3  2023-01-11 00:00:00  ABC      return_before_session   0
 2  2023-01-11 00:00:00  ABC      return_in_session       0
 5  2023-01-12 00:00:00  ABC      return_before_session   0
 4  2023-01-12 00:00:00  ABC      return_in_session       0
 7  2023-01-13 00:00:00  ABC      return_before_session  -0.119801
 6  2023-01-13 00:00:00  ABC      return_in_session       0.167054
 9  2023-01-16 00:00:00  ABC      return_before_session  -0.0312525
 8  2023-01-16 00:00:00  ABC      return_in_session       0


In [ ]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.3/907.3 KB 13.6 MB/s eta 0:00:00


In [ ]:
from arch import arch_model
import seaborn as sns
import matplotlib.pyplot as plt
def volatility(ma_ck):
  data = price_stack[price_stack['ma_ck']==ma_ck]
  model = arch_model(data['return'], mean='Zero', vol='ARCH', p=1)
  model_fit = model.fit()
  data['volatility']=model_fit.conditional_volatility 
  return data

In [ ]:
vol_df = pd.DataFrame(columns=volatility(price_stack['ma_ck'].unique()[0]).columns)
vol_df

Iteration:      1,   Func. Count:      4,   Neg. LLF: -43.7016179240237
Iteration:      2,   Func. Count:     10,   Neg. LLF: -86.90821669538018
Iteration:      3,   Func. Count:     15,   Neg. LLF: -89.48249380911157
Iteration:      4,   Func. Count:     18,   Neg. LLF: -89.48253473380213
Iteration:      5,   Func. Count:     20,   Neg. LLF: -89.48253473383589
Optimization terminated successfully    (Exit mode 0)
            Current function value: -89.48253473380213
            Iterations: 5
            Function evaluations: 20
            Gradient evaluations: 5


/usr/local/lib/python3.8/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002849. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
<ipython-input-16-b942dc21cacd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['volatility']=model_fit.conditional_volatility


Empty DataFrame
Columns: [date, ma_ck, session, return, volatility]
Index: []

In [ ]:
for i in price_stack['ma_ck'].unique():
  try:
    vol_df = vol_df.append(volatility(i))
  except:
    print(i)
    break

In [ ]:
vol_df

date ma_ck                session   return  volatility
1     2023-01-10   ABC  return_before_session  0.00000    0.054232
0     2023-01-10   ABC      return_in_session  0.00000    0.046948
3     2023-01-11   ABC  return_before_session  0.00000    0.046948
2     2023-01-11   ABC      return_in_session  0.00000    0.046948
5     2023-01-12   ABC  return_before_session  0.00000    0.046948
...          ...   ...                    ...      ...         ...
12750 2023-02-22   XPH      return_in_session  0.00000    0.027804
12753 2023-02-23   XPH  return_before_session  0.00000    0.027804
12752 2023-02-23   XPH      return_in_session  0.00000    0.027804
12755 2023-02-24   XPH  return_before_session -0.02353    0.027804
12754 2023-02-24   XPH      return_in_session  0.00000    0.027804

[12756 rows x 5 columns]

# Ghep bang gia va NLP

In [12]:
from datetime import datetime, timedelta
title['new_date'] = title['date']
title['weekend'] = False
title['weekend'][title['weekday']=='Friday'] = title[title['weekday']=='Friday']['hour']>16
title['new_date'][title['hour']>16] = title[title['hour']>16]['new_date']+timedelta(days=1)
title['new_date'][title['weekend']==True] = title[title['weekend']==True]['new_date']+timedelta(days=2)
title['session'] = ['return_in_session' if (x>8) and (x<16) else 'return_before_session' for x in title['hour']]
print(tabulate(title.head(10), headers='keys'))

    date                   hour  ma_ck    title                                                                         weekday    title_new                                            new_date             weekend    session
--  -------------------  ------  -------  ----------------------------------------------------------------------------  ---------  ---------------------------------------------------  -------------------  ---------  ---------------------
 0  2023-02-24 00:00:00    17.5  ABC      ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán trưởng từ 24/02/2023            Friday     miễn_nhiệm dương kế_toán_trưởng                      2023-02-27 00:00:00  True       return_before_session
 1  2023-02-24 00:00:00    17.2  ABC      ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng cổ đông thường niên năm 2023   Friday     tham_dự đại_hội_đồng cổ_đông                         2023-02-27 00:00:00  True       return_before_session
 2  2023-02-20 00:00:00    15.3  ABC      ABC: Bổ nhiệm bà Hồ 

<ipython-input-12-40d143527722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title['weekend'][title['weekday']=='Friday'] = title[title['weekday']=='Friday']['hour']>16
<ipython-input-12-40d143527722>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title['new_date'][title['hour']>16] = title[title['hour']>16]['new_date']+timedelta(days=1)
<ipython-input-12-40d143527722>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title['new_d

In [20]:
print(tabulate(title.head(10)[['date', 'hour', 'ma_ck', 'title']], headers='keys'))

    date                   hour  ma_ck    title
--  -------------------  ------  -------  ----------------------------------------------------------------------------
 0  2023-02-24 00:00:00    17.5  ABC      ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán trưởng từ 24/02/2023
 1  2023-02-24 00:00:00    17.2  ABC      ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng cổ đông thường niên năm 2023
 2  2023-02-20 00:00:00    15.3  ABC      ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công bố thông tin từ 22/02/2023
 3  2023-01-31 00:00:00    17.2  ABC      ABC: Báo cáo quản trị công ty năm 2022
 4  2023-01-30 00:00:00    15.5  ACM      ACM: Đính chính Báo cáo quản trị công ty năm 2022
 5  2023-02-17 00:00:00    15    AGF      AGF: 14.3.2023, ngày GDKHQ Tham dự đại hội đồng cổ đông thường niên năm 2023
 6  2023-02-13 00:00:00    16.5  AGF      AGF: Nghị quyết HĐQT thống nhất tổ chức ĐHĐCĐ TN 2023
 7  2023-02-10 00:00:00    14.4  AGF      AGF: Nghị quyết HĐQT v/v tổ chức ĐHĐCĐ TN 2023
 8  2023-01-1

In [21]:
print(tabulate(title.head(10)[['weekday', 'title_new', 'new_date', 'weekend', 'session']], headers='keys'))

    weekday    title_new                                            new_date             weekend    session
--  ---------  ---------------------------------------------------  -------------------  ---------  ---------------------
 0  Friday     miễn_nhiệm dương kế_toán_trưởng                      2023-02-27 00:00:00  True       return_before_session
 1  Friday     tham_dự đại_hội_đồng cổ_đông                         2023-02-27 00:00:00  True       return_before_session
 2  Monday     bổ_nhiệm hồ thị hà giữ chức người công_bố thông_tin  2023-02-20 00:00:00  False      return_in_session
 3  Tuesday    báo_cáo quản_trị công_ty                             2023-02-01 00:00:00  False      return_before_session
 4  Monday     đính_chính báo_cáo quản_trị                          2023-01-30 00:00:00  False      return_in_session
 5  Friday     tham_dự đại_hội_đồng cổ_đông                         2023-02-17 00:00:00  False      return_in_session
 6  Monday     nghị_quyết hđqt thống_nhất tổ_chức 

In [16]:
price_stack = price_stack.rename(columns={'date':'new_date'})
df_delta = title.merge(price_stack, on=['new_date', 'ma_ck', 'session'], how='left')
print(tabulate(df_delta.head(10), headers='keys'))

    date                   hour  ma_ck    title                                                                         weekday    title_new                                            new_date             weekend    session                     return
--  -------------------  ------  -------  ----------------------------------------------------------------------------  ---------  ---------------------------------------------------  -------------------  ---------  ---------------------  -----------
 0  2023-02-24 00:00:00    17.5  ABC      ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán trưởng từ 24/02/2023            Friday     miễn_nhiệm dương kế_toán_trưởng                      2023-02-27 00:00:00  True       return_before_session  nan
 1  2023-02-24 00:00:00    17.2  ABC      ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng cổ đông thường niên năm 2023   Friday     tham_dự đại_hội_đồng cổ_đông                         2023-02-27 00:00:00  True       return_before_session  nan
 2  2023-02-

In [17]:
df_delta = df_delta.drop_duplicates(ignore_index=True)
df_delta = df_delta.dropna().reset_index(drop=True)
df_delta                                    

date  hour ma_ck                                              title  \
0   2023-02-20  15.3   ABC  ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...   
1   2023-01-31  17.2   ABC             ABC: Báo cáo quản trị công ty năm 2022   
2   2023-01-30  15.5   ACM  ACM: Đính chính Báo cáo quản trị công ty năm 2022   
3   2023-02-17  15.0   AGF  AGF: 14.3.2023, ngày GDKHQ Tham dự đại hội đồn...   
4   2023-02-13  16.5   AGF  AGF: Nghị quyết HĐQT thống nhất tổ chức ĐHĐCĐ ...   
..         ...   ...   ...                                                ...   
698 2023-01-18   8.1   X77             X77: Báo cáo quản trị công ty năm 2022   
699 2023-02-09  13.2   XPH  XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...   
700 2023-02-09  13.2   XPH  XPH: Quyết định duy trì diện bị hạn chế giao d...   
701 2023-02-03   8.5   XPH  XPH: Giải trình liên quan đến Báo cáo tài chín...   
702 2023-02-02  13.4   XPH             XPH: Báo cáo quản trị công ty năm 2022   

       weekday                                          title_new   new_date  \
0       Monday  bổ_nhiệm hồ thị hà giữ chức người công_bố thôn... 2023-02-20   
1      Tuesday                           báo_cáo quản_trị công_ty 2023-02-01   
2       Monday                        đính_chính báo_cáo quản_trị 2023-01-30   
3       Friday                       tham_dự đại_hội_đồng cổ_đông 2023-02-17   
4       Monday        nghị_quyết hđqt thống_nhất tổ_chức đhđcđ tn 2023-02-14   
..         ...                                                ...        ...   
698  Wednesday                           báo_cáo quản_trị công_ty 2023-01-18   
699   Thursday  bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ... 2023-02-09   
700   Thursday  quyết_định duy_trì diện bị hạn_chế giao_dịch h... 2023-02-09   
701     Friday     giải_trình liên_quan báo_cáo tài_chính niên độ 2023-02-03   
702   Thursday                           báo_cáo quản_trị công_ty 2023-02-02   

    weekend                session   return  
0     False      return_in_session  0.00000  
1     False  return_before_session  0.00000  
2     False      return_in_session  0.00000  
3     False      return_in_session  0.03774  
4     False  return_before_session  0.00000  
..      ...                    ...      ...  
698   False      return_in_session  0.00000  
699   False      return_in_session  0.00000  
700   False      return_in_session  0.00000  
701   False      return_in_session  0.00000  
702   False      return_in_session  0.00000  

[703 rows x 10 columns]

In [19]:
df_delta_sub = df_delta
df_delta_sub['title_new'] = df_delta_sub['title_new']+' '
df_final = df_delta_sub[['ma_ck', 'title_new', 'new_date', 'session', 'return']].groupby(['new_date', 'ma_ck', 'session', 'return']).sum().reset_index()
df_final['title_new'] = df_final['title_new'].str[:-1]
print(tabulate(df_final.head(10), headers='keys', ))

    new_date             ma_ck    session                     return  title_new
--  -------------------  -------  ---------------------  -----------  ------------------------------------------------------------------------------------------------------------
 0  2023-01-10 00:00:00  BHT      return_in_session       0           tổ_chức đại_hội_đồng cổ_đông
 1  2023-01-10 00:00:00  HAG      return_before_session  -0.00883008  hoàng anh gia_lai lãi trở_lại sau bầu đức hào_phóng thưởng đậm nhân_viên mức cao thuộc người ' trồng chuối '
 2  2023-01-10 00:00:00  PET      return_in_session      -0.00762393  thực_hiện quyền nhận cổ_tức cổ_phiếu tỷ_lệ
 3  2023-01-10 00:00:00  PLX      return_in_session       0.0486545   sau khi hạ tiêu petrolimex bất_ngờ báo lãi thuế gần a
 4  2023-01-10 00:00:00  PRT      return_in_session      -0.0175443   thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông
 5  2023-01-10 00:00:00  SCO      return_in_session       0           báo_cáo quản_trị công_ty
 6  2023-01-10

In [23]:
print(tabulate(df_final.head(10)[['new_date', 'ma_ck', 'session', 'return']], headers='keys'))

    new_date             ma_ck    session                     return
--  -------------------  -------  ---------------------  -----------
 0  2023-01-10 00:00:00  BHT      return_in_session       0
 1  2023-01-10 00:00:00  HAG      return_before_session  -0.00883008
 2  2023-01-10 00:00:00  PET      return_in_session      -0.00762393
 3  2023-01-10 00:00:00  PLX      return_in_session       0.0486545
 4  2023-01-10 00:00:00  PRT      return_in_session      -0.0175443
 5  2023-01-10 00:00:00  SCO      return_in_session       0
 6  2023-01-10 00:00:00  TDH      return_before_session  -0.0650325
 7  2023-01-11 00:00:00  ATG      return_before_session   0
 8  2023-01-11 00:00:00  BT6      return_before_session   0
 9  2023-01-11 00:00:00  HTT      return_in_session       0


In [24]:
print(tabulate(df_final.head(10)[['title_new']], headers='keys'))

    title_new
--  ------------------------------------------------------------------------------------------------------------
 0  tổ_chức đại_hội_đồng cổ_đông
 1  hoàng anh gia_lai lãi trở_lại sau bầu đức hào_phóng thưởng đậm nhân_viên mức cao thuộc người ' trồng chuối '
 2  thực_hiện quyền nhận cổ_tức cổ_phiếu tỷ_lệ
 3  sau khi hạ tiêu petrolimex bất_ngờ báo lãi thuế gần a
 4  thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông
 5  báo_cáo quản_trị công_ty
 6  thông_qua chủ_trương giao_dịch bên có
 7  bổ_nhiệm nguyễn thanh_tùng giữ chức phụ_trách kế_toán
 8  báo_cáo quản_trị công_ty
 9  đào văn_chiến chủ_tịch hđqt mua cp


In [ ]:
df_final.to_csv('data/input_table.csv', index=False)